In [187]:
# necessary imports
import time as time_ #make sure we don't override time

import pandas
import numpy
import warnings
from collections import Counter, defaultdict

In [188]:
def ignore_warn(*args, **kwargs):
    pass
warnings.warn = ignore_warn

def millis():
    return int(round(time_.time() * 1000))

# Variable declarations
FILE_PATH = "Filtered_Data.csv"
SEPARATOR = ","

CLICK_THRESHOLD_FOR_FEATURE = 1000
CLICK_THRESHOLD_FOR_DOMAIN = 50
CONVERSION_THRESHOLD_FOR_FEATURE = 1000
CONVERSION_THRESHOLD_FOR_DOMAIN = 5

MAX_ROWS = 1500
MISCELLANEOUS = 'MISC'
FEATURE_NAMES = ['ad_advertiser_id', 'analyzer_name', 'click_browser', 'enriched_derived_device', 'enriched_derived_os', 'raw_sync_partner_id']

web_urls = ['www.msn.com', 'www.collegescholarships.org', 'gotquestion.com', 'homebidz.co', 'searchads.local.com', 'www.dg,coupon.com']
start_time = millis()
# imported the CSV data
dataframe = pandas.read_csv(FILE_PATH, sep = SEPARATOR)
end_time = millis()
print ('Took ', end_time - start_time, 'ms')

# Drop the unused column
if 'Unnamed: 0' in dataframe.columns:
    dataframe.drop('Unnamed: 0', axis = 1, inplace = True)

# Remove the same prefix from all columns names
dataframe.columns  = [ column_name.split('.')[1] for column_name in dataframe.columns ]

dataframe = dataframe.loc[dataframe['enriched_publisher_domain'].isin(web_urls)]
pandas.set_option('display.max_rows', MAX_ROWS)
dataframe.head()

Took  1555 ms


,ts,impression_system_date,hour,conversion_status,cc_id_null_ind,ad_id,click_id,cc_id,hour_of_day,raw_publisher_id,...,click_conf_gmt_offset,click_state,click_city,ad_bid,ad_cpa_goal,ad_market_place_id,click_click_status,kmean_category_name,partner_id,version
0,20190106,2019-01-06,1,1,0,138154,646739319951006038,546739320882001606,20,8CUDMGV55,...,-500,ga,atlanta,0.150000,NaN,1027,1,missing,8PRHGG6T9,1
1,20190106,2019-01-06,1,1,0,138154,646739319951006038,546739320882001606,20,8CUDMGV55,...,-500,ga,atlanta,0.150000,NaN,1027,1,missing,8PRHGG6T9,1
2,20190106,2019-01-06,2,0,0,138154,646740715393002311,346740715723006848,20,8CUDMGV55,...,-600,al,birmingham,0.150000,NaN,1027,1,missing,8PRHGG6T9,1
3,20190106,2019-01-06,4,0,0,138154,246748175931002645,446748176329007771,23,8CUDMGV55,...,-500,in,ft wayne,0.150000,NaN,1027,1,missing,8PRHGG6T9,1
6,20190106,2019-01-06,7,0,0,138155,146759863823002600,446759863997003939,23,8CUDMGV55,...,-800,ca,modesto,0.529265,NaN,1027,1,missing,8PRHGG6T9,1


In [189]:
# removing all data having single values in entire columns
start_time = millis()
list_of_single_value_columns = []
for column in dataframe.columns:
    if(dataframe[column].nunique() == 1):
        dataframe.drop(column, axis = 1, inplace = True)
        list_of_single_value_columns.append(column)

end_time = millis()
print ('Took ', end_time - start_time, 'ms')

print (list_of_single_value_columns)

Took  355 ms
['cc_id_null_ind', 'enriched_country', 'serving_key', 'ad_partner_id', 'ad_market_place_id', 'click_click_status', 'kmean_category_name', 'version']


In [190]:
start_time = millis()

# Remove the timestamp data and impression system date data - results not dependent on the time frame
dataframe.drop('ts', axis = 1, inplace = True)
dataframe.drop('impression_system_date', axis = 1, inplace = True)

# Server hour do not matters
dataframe.drop('hour', axis = 1, inplace = True)

# Data have only 3 values of *au* region, and conversion status 0 for all of them
# dataframe.drop('enriched_country', axis = 1, inplace = True)

# Data is similar to cvr_base_data_table.click_city
dataframe.drop('impression_city', axis = 1, inplace = True)

# Ad-Bid is the cost which is irrespecti   ve of conversion from user side
dataframe.drop('ad_bid', axis = 1, inplace = True)

# Ad CPA goal do not affects the conversion status
dataframe.drop('ad_cpa_goal', axis = 1, inplace = True)

# Click Conf GMT Offset do not affects the conversion status
dataframe.drop('click_conf_gmt_offset', axis = 1, inplace = True)

# Date Time do not affects and is too granuler to looku[, thus do not affects the conversion status
dataframe.drop('click_user_datetime', axis = 1, inplace = True)

# Publisher Domain do not affects the conversion status
# dataframe.drop('enriched_publisher_domain', axis = 1, inplace = True)

# Impression ISP Name do not affects the conversion status
dataframe.drop('impression_isp_name', axis = 1, inplace = True)

# below data is too granular to predict something
dataframe.drop('ad_keyword', axis = 1, inplace = True)
dataframe.drop('click_city', axis = 1, inplace = True)
dataframe.drop('raw_sub_publisher_id', axis = 1, inplace = True)
dataframe.drop('category_name', axis = 1, inplace = True)
dataframe.drop('click_id', axis = 1, inplace = True)
dataframe.drop('cc_id', axis = 1, inplace = True)
dataframe.drop('raw_publisher_id', axis = 1, inplace = True)
dataframe.drop('raw_sub_sub_publisher_id', axis = 1, inplace = True)

end_time = millis()
print ('Took ', end_time - start_time, 'ms')

Took  237 ms


In [191]:
# Function for aggregation on basis of click, conversion and CVR(conversion ratio)
func = {'conversion_status' : {
    'conversion' : 'sum',
    'click' : 'count'
}}

In [192]:
total = dataframe.shape[0]

# Iterate through above columns list and 'misc' the data which is too sparse to use 
# for classification or modelling
def mark_msic_values(dataframe, feature_name, click_threshold, conversion_threshold):
    dfg = dataframe.groupby(feature_name).agg(func).reset_index()
    dfg.columns = dfg.columns.droplevel(0)
    dfg['cvr'] = dfg['conversion'] / dfg['click']
    dfg['click percentage of total data'] = dfg['click'] / total * 100
    declareMisc = set()
    for index, row in dfg.iterrows():
        if(row['click'] >= click_threshold and row['conversion'] >= 5):
            continue
        declareMisc.add(row[''])
    print (declareMisc)
    for vals in declareMisc:
        dataframe.loc[dataframe[feature_name] == vals, feature_name] = MISCELLANEOUS
    return dataframe

# Iterate through above columns list and 'misc' the data which is too sparse to use 
# for classification or modelling
start_time = millis()
for col in FEATURE_NAMES:
    dataframe = mark_msic_values(dataframe, col, CLICK_THRESHOLD_FOR_FEATURE, CONVERSION_THRESHOLD_FOR_FEATURE)
dataframe = mark_msic_values(dataframe, col, CLICK_THRESHOLD_FOR_DOMAIN, CONVERSION_THRESHOLD_FOR_DOMAIN)
end_time = millis()
print ('Took ', end_time - start_time, 'ms')

{1633.0, 1351.0, 1641.0, 1517.0, 1492.0, 1461.0, 1375.0}
{'categoryanalyser', 'forcedanalyser', 'contextualanalyser', 'broadanalyserv2', 'nofilteranalyser'}
{'blackberry,browser', 'mozilla,like,unknown', 'ie,mobile', 'applewebkit', 'opera,mini', 'maxthon', 'chromium', 'ucbrowser', 'unknown', 'android,browser', 'opera', 'nokia', 'silk'}
{'unknown,device'}
{'linux', 'unknown,os'}
set()
set()
Took  1106 ms


In [193]:
# Now, we need to remove the outliers for certain feature colums
# We will use standard method for removing the outliers
# called as Tukeys method for determining the data need to be removed
# from the original data
# As outliers should be never considered for analysis

In [194]:
# classfiy Miscellaneous points
def get_index_of_outliers(dataframe, feature_name, outlier_indexes):
    dfg = dataframe.groupby(feature_name).agg(func).reset_index()
    dfg.columns = dfg.columns.droplevel(0)
    dfg['cvr'] = dfg['conversion'] / dfg['click']
    range1 = numpy.percentile(dfg['cvr'], 25)
    range2 = numpy.percentile(dfg['cvr'], 75)
    step = 1.5 * (range2 - range1)
    lower_limit = range1 - step
    upper_limit = range2 + step
    indexes = dfg[(dfg['cvr'] < lower_limit) | (dfg['cvr'] > upper_limit)].index
    
    if(len(indexes) == 0):
        return outlier_indexes
    
    label_names = set()
    for index in indexes:
        label_names.add(dfg.iloc[index][''])
    for col_value in label_names:
        indexes_in_original_dataframe = dataframe[dataframe[col] == col_value].index.tolist()
        for index_in_df in indexes_in_original_dataframe:
            outlier_indexes.add(index_in_df)
    return outlier_indexes

In [195]:
final_indexes = set()

start_time = millis()
for col in FEATURE_NAMES:
    final_indexes = get_index_of_outliers(dataframe, col, final_indexes)
end_time = millis()
print ('Took ', end_time - start_time, 'ms')

start_time = millis()
final_indexes = list(final_indexes)
dataframe = dataframe.drop(final_indexes, axis = 0).reset_index(drop = True)
end_time = millis()
print ('Took ', end_time - start_time, 'ms')

print (final_indexes)

Took  150 ms
Took  21 ms
[0, 1, 2, 3, 6, 7, 8, 12, 14, 18, 19, 21, 22, 24, 25, 26, 28, 29, 30, 31, 32, 34, 36, 37, 38, 39, 40, 42, 43, 44, 45, 50, 51, 52, 53, 54, 55, 56, 58, 59, 60, 61, 65, 66, 68, 70, 71, 72, 73, 74, 76, 77, 78, 79, 81, 82, 83, 84, 85, 86, 88, 89, 90, 91, 92, 95, 102, 106, 116, 118, 121, 133, 139, 144, 146, 156, 157, 161, 166, 168, 171, 194, 196, 199, 200, 201, 202, 210, 230, 233, 237, 239, 254, 258, 262, 266, 268, 270, 276, 293, 297, 302, 308, 314, 315, 316, 322, 326, 330, 338, 345, 352, 354, 361, 370, 372, 374, 375, 376, 378, 388, 389, 391, 394, 400, 404, 407, 408, 434, 438, 440, 447, 449, 450, 455, 457, 467, 470, 474, 477, 482, 484, 485, 492, 499, 501, 510, 514, 523, 534, 536, 538, 539, 541, 543, 552, 553, 556, 558, 590, 591, 592, 600, 607, 608, 612, 616, 622, 625, 628, 630, 631, 632, 636, 637, 642, 650, 652, 654, 659, 662, 673, 674, 676, 678, 681, 688, 692, 705, 715, 719, 723, 731, 734, 736, 754, 760, 762, 763, 765, 769, 777, 780, 781, 782, 785, 787, 790, 795, 79

In [196]:
epsilon = float(.1)

def equal_dataframes(df1, df2):
    if(len(df1) != len(df2)):
        return False
    for index, row in df1.iterrows():
        if(abs(df1.iloc[index]['cvr'] - df2.iloc[index]['cvr']) > epsilon):
            return False
    return True

def calcualte_mean(current_dataframe, idx):
    feature_map_sum = defaultdict(float)
    feature_map_count = defaultdict(float)
    feature_map_mean = {}
    idx = int(idx)
    for index, row in current_dataframe.iterrows():
        feature_map_count[row[idx]] = feature_map_count[row[idx]] + 1.0000000
        feature_map_sum[row[idx]] = feature_map_sum[row[idx]] + row['cvr']
    for key in feature_map_count.keys():
        try:
            feature_map_mean[key] = (1.0 / ((feature_map_sum[key] / feature_map_count[key])))
        except:
            feature_map_mean[key] = 0.0
            print ('Zero Value dectected', key)
    return feature_map_mean
    
def get_normalized_score(original_dataframe, main_feature, bias_feature):
    ct = int(0)
    df_set = []
    original_dfg = original_dataframe.groupby([main_feature, bias_feature]).agg(func).reset_index()
    original_dfg.columns = original_dfg.columns.droplevel(0)
    original_dfg['cvr'] = original_dfg['conversion'] / original_dfg['click']
#     print ('Inside function')
    current_dfg = original_dfg.copy()
    last_dfg = pandas.DataFrame()
#     print ('Passed the last and current')
#     print (original_dfg.head(), '\n\n')
    while(True):
        start_time = millis()
        
        last_dfg = current_dfg.copy()
#         print ('Inside while loop')
        
        mainF_map_mean = calcualte_mean(current_dfg, 0)
        biasF_map_mean = calcualte_mean(current_dfg, 1)
        
        for index, row in current_dfg.iterrows():
            current_dfg.loc[index, 'cvr'] = float(original_dfg.loc[index]['cvr'] * mainF_map_mean[original_dfg.loc[index][0]])
        
        df_set.append(current_dfg.copy())
        if(equal_dataframes(current_dfg, last_dfg)):
            return df_set, calcualte_mean(current_dfg, 0)
        if(ct == 100):
            return df_set, calcualte_mean(current_dfg, 0)
        ct += 1
        print (ct)
        end_time = millis()
        print ('Took ', end_time - start_time, 'ms')
    return df_set, calcualte_mean(current_dfg, 0)

In [197]:
dataframe.shape
start_time = millis()
df_set, map_val = get_normalized_score(dataframe, 'enriched_publisher_domain', 'ad_advertiser_id')
end_time = millis()
print ('Took ', end_time - start_time, 'ms')

# for w in sorted(map_val, key = map_val.get, reverse = True):
#     print (w, map_val[w])

Zero Value dectected 1387
Zero Value dectected 1521
Zero Value dectected 1640
1
Took  124 ms
Zero Value dectected 1387
Zero Value dectected 1521
Zero Value dectected 1640
2
Took  130 ms
Zero Value dectected 1387
Zero Value dectected 1521
Zero Value dectected 1640
3
Took  104 ms
Zero Value dectected 1387
Zero Value dectected 1521
Zero Value dectected 1640
4
Took  105 ms
Zero Value dectected 1387
Zero Value dectected 1521
Zero Value dectected 1640
5
Took  102 ms
Zero Value dectected 1387
Zero Value dectected 1521
Zero Value dectected 1640
6
Took  101 ms
Zero Value dectected 1387
Zero Value dectected 1521
Zero Value dectected 1640
7
Took  104 ms
Zero Value dectected 1387
Zero Value dectected 1521
Zero Value dectected 1640
8
Took  110 ms
Zero Value dectected 1387
Zero Value dectected 1521
Zero Value dectected 1640
9
Took  120 ms
Zero Value dectected 1387
Zero Value dectected 1521
Zero Value dectected 1640
10
Took  104 ms
Zero Value dectected 1387
Zero Value dectected 1521
Zero Value dectec

89
Took  153 ms
Zero Value dectected 1387
Zero Value dectected 1521
Zero Value dectected 1640
90
Took  107 ms
Zero Value dectected 1387
Zero Value dectected 1521
Zero Value dectected 1640
91
Took  100 ms
Zero Value dectected 1387
Zero Value dectected 1521
Zero Value dectected 1640
92
Took  119 ms
Zero Value dectected 1387
Zero Value dectected 1521
Zero Value dectected 1640
93
Took  94 ms
Zero Value dectected 1387
Zero Value dectected 1521
Zero Value dectected 1640
94
Took  117 ms
Zero Value dectected 1387
Zero Value dectected 1521
Zero Value dectected 1640
95
Took  141 ms
Zero Value dectected 1387
Zero Value dectected 1521
Zero Value dectected 1640
96
Took  98 ms
Zero Value dectected 1387
Zero Value dectected 1521
Zero Value dectected 1640
97
Took  100 ms
Zero Value dectected 1387
Zero Value dectected 1521
Zero Value dectected 1640
98
Took  127 ms
Zero Value dectected 1387
Zero Value dectected 1521
Zero Value dectected 1640
99
Took  96 ms
Zero Value dectected 1387
Zero Value dectected 

In [198]:
if(equal_dataframes(df_set[8], df_set[10])):
    print ('yes')
final_df = dataframe.groupby('enriched_publisher_domain').agg(func).reset_index()
final_df.columns = final_df.columns.droplevel(0)
final_df['cvr'] = final_df['conversion'] / final_df['click']
final_df['Actual_score'] = float(0.0)


for index, row in final_df.iterrows():
    final_df.at[index, 'Actual_score'] = float(map_val[final_df.loc[index][0]])
final_df.sort_values(by = ['Actual_score'], ascending = False)

yes


In [199]:
# dfg = dataframe.groupby()